In [ ]:
import numpy as np
import pandas as pd
import os

def tokenize(text):
    """Tokeniza o texto convertendo-o para minúsculas e dividindo por espaços."""
    return text.lower().split()

def vectorize(text, vocab):
    """Converte um texto em um vetor de bag-of-words usando o vocabulário fornecido."""
    vector = np.zeros(len(vocab), dtype=np.float32)
    for token in tokenize(text):
        if token in vocab:
            vector[vocab[token]] += 1
    return vector

def predict_logistic(X, weights, bias, threshold=0.5):
    """Faz predições com o modelo de regressão logística."""
    z = np.dot(X, weights) + bias
    prob = 1 / (1 + np.exp(-z))
    return (prob >= threshold).astype(int)

def main():
    # Caminho do modelo treinado
    model_path = "data/trained_logistic_chunked.npz"
    if not os.path.exists(model_path):
        raise FileNotFoundError(f"Modelo não encontrado em: {model_path}")

    # Carregar os parâmetros do modelo e o vocabulário
    model_data = np.load(model_path, allow_pickle=True)
    weights = model_data["weights"]
    bias = model_data["bias"]
    vocab_tokens = model_data["vocab_tokens"].tolist()
    vocab_indices = model_data["vocab_indices"].tolist()

    # Reconstruir o vocabulário (token -> índice)
    vocab = {token: idx for token, idx in zip(vocab_tokens, vocab_indices)}
    print(f"Vocabulário reconstituído: {len(vocab)} tokens.")

    # Caminho do CSV de input (com delimitador ";")
    input_csv = "data/dataset2_inputs.csv"
    if not os.path.exists(input_csv):
        raise FileNotFoundError(f"Arquivo de input não encontrado: {input_csv}")

    # Ler o CSV forçando delimitador=";", ignorando linhas ruins
    # Se o arquivo tiver alguma linha com colunas a mais, elas serão puladas
    df_input = pd.read_csv(
        input_csv,
        delimiter=";",
        engine="python",
        on_bad_lines="skip"
    )

    # Verificar se as colunas são "ID" e "Text"
    if "ID" not in df_input.columns or "Text" not in df_input.columns:
        raise KeyError("O arquivo dataset2_inputs.csv deve conter as colunas 'ID' e 'Text'.")

    ids = df_input["ID"].tolist()
    texts = df_input["Text"].astype(str).tolist()
    print(f"Dataset de input carregado com {len(texts)} textos (linhas válidas).")
    print("Colunas lidas:", df_input.columns.tolist())

    # Converter os textos para vetores utilizando o vocabulário
    X_input = np.array([vectorize(text, vocab) for text in texts], dtype=np.float32)
    print("Forma de X_input:", X_input.shape)

    # Fazer as predições usando o modelo treinado
    preds = predict_logistic(X_input, weights, bias)
    # Mapear 0 -> "Human", 1 -> "AI"
    predicted_labels = ["AI" if p == 1 else "Human" for p in preds.flatten()]

    # Criar o DataFrame de saída e salvar as predições
    df_output = pd.DataFrame({
        "ID": ids,
        "Predicted": predicted_labels
    })
    output_csv = "data/dataset2_predictions.csv"
    df_output.to_csv(output_csv, index=False, sep="\t")
    print(f"\nArquivo de predições gerado: {output_csv}")
    print(df_output.head())

if __name__ == "__main__":
    main()